In [1]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ast
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.agent import predict_position
from src.update_files import update_state, update_portfolio
from src.position_sizing import portfolio_calculator, amount_calculator
from src.backtester import data_handler
from src.utils import get_config, read_file

In [4]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

['ETH/USDT',
 'BTC/USDT',
 'BNB/USDT',
 'SOL/USDT',
 'ADA/USDT',
 'LINK/USDT',
 'XRP/USDT',
 'LTC/USDT',
 'XLM/USDT']

In [5]:
test_data = read_file.read_merged_test_data()
test_data

,"('open', 'ETH')","('high', 'ETH')","('low', 'ETH')","('close', 'ETH')","('volume', 'ETH')","('rsi', 'ETH')","('sma-50', 'ETH')","('sma-100', 'ETH')","('sma-200', 'ETH')","('ema-50', 'ETH')",...,"('volume', 'XLM')","('rsi', 'XLM')","('sma-50', 'XLM')","('sma-100', 'XLM')","('sma-200', 'XLM')","('ema-50', 'XLM')","('ema-100', 'XLM')","('ema-200', 'XLM')","('atr', 'XLM')","('adx', 'XLM')"
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-01 00:00:00,0.329613,0.332409,0.330081,0.331737,0.045879,0.558219,0.333974,0.343871,0.369282,0.337550,...,0.003740,0.434468,0.307526,0.321831,0.343250,0.305276,0.321090,0.339656,0.051778,0.275755
2025-04-01 01:00:00,0.331719,0.330275,0.330781,0.330504,0.023141,0.529522,0.333967,0.343435,0.369090,0.337520,...,0.001500,0.423610,0.307397,0.321409,0.343105,0.305071,0.320862,0.339464,0.050619,0.268369
2025-04-01 02:00:00,0.330489,0.331613,0.331528,0.333386,0.021161,0.587390,0.334023,0.343052,0.368908,0.337606,...,0.005161,0.513645,0.307366,0.321039,0.342987,0.305059,0.320734,0.339324,0.050824,0.246534
2025-04-01 03:00:00,0.333370,0.331990,0.333454,0.332415,0.021518,0.563701,0.334023,0.342668,0.368722,0.337650,...,0.002176,0.481430,0.307253,0.320693,0.342836,0.304980,0.320574,0.339167,0.050246,0.225046
2025-04-01 04:00:00,0.332399,0.331864,0.333207,0.333109,0.015722,0.577911,0.334019,0.342289,0.368545,0.337720,...,0.001354,0.504186,0.307108,0.320354,0.342711,0.304952,0.320443,0.339025,0.048941,0.203206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30 19:00:00,0.838104,0.846052,0.839065,0.847804,0.060454,0.543642,0.858427,0.849183,0.860952,0.859990,...,0.003947,0.532482,0.465912,0.473415,0.482209,0.460014,0.473816,0.494031,0.055621,0.205222
2025-09-30 20:00:00,0.847763,0.852013,0.847896,0.855620,0.042644,0.628021,0.859118,0.849649,0.860955,0.860538,...,0.003175,0.562437,0.466126,0.473603,0.482282,0.460238,0.473958,0.494073,0.054700,0.204114
2025-09-30 21:00:00,0.855579,0.852914,0.848765,0.846172,0.040228,0.511478,0.859629,0.849894,0.860892,0.860686,...,0.004165,0.483560,0.466227,0.473683,0.482306,0.460251,0.473991,0.494060,0.055639,0.196806


In [6]:
update_state.set_state(test_data, 100000)
state = read_file.read_state()
state

{'timestep': '2025-04-01 00:00:00',
 'cash': 100000,
 'returns': 0,
 'drawdown': 0,
 'sharpe ratio': 0}

In [7]:
candle = data_handler.fetch_data(test_data)
candle

{'ADA': {'adx': 0.2193063202286422,
  'atr': 0.0493882016057434,
  'close': 0.1907295212049226,
  'ema-100': 0.206824163245561,
  'ema-200': 0.2169855960673466,
  'ema-50': 0.1994327011436139,
  'high': 0.1920164718384697,
  'low': 0.1932285385142919,
  'open': 0.1904660509435468,
  'rsi': 0.536418109655395,
  'sma-100': 0.2061723580663051,
  'sma-200': 0.2207249308392412,
  'sma-50': 0.1973433319510942,
  'volume': 0.0127223493665695},
 'BNB': {'adx': 0.115147560229091,
  'atr': 0.1219564490252879,
  'close': 0.8375729858653909,
  'ema-100': 0.8870525182073266,
  'ema-200': 0.9182748567281164,
  'ema-50': 0.8591913185312718,
  'high': 0.8356817095772694,
  'low': 0.8428214210201668,
  'open': 0.8341523164493119,
  'rsi': 0.6002544031058116,
  'sma-100': 0.8720682330813073,
  'sma-200': 0.9214852602852838,
  'sma-50': 0.8521636947536342,
  'volume': 0.0024030637997116},
 'BTC': {'adx': 0.1954733472661476,
  'atr': 0.2641607961662952,
  'close': 1.1437067869989446,
  'ema-100': 1.194902

In [8]:
candle = predict_position.predict(candle)
candle

{'ADA': {'adx': 0.2193063202286422,
  'atr': 0.0493882016057434,
  'close': 0.1907295212049226,
  'ema-100': 0.206824163245561,
  'ema-200': 0.2169855960673466,
  'ema-50': 0.1994327011436139,
  'high': 0.1920164718384697,
  'low': 0.1932285385142919,
  'open': 0.1904660509435468,
  'rsi': 0.536418109655395,
  'sma-100': 0.2061723580663051,
  'sma-200': 0.2207249308392412,
  'sma-50': 0.1973433319510942,
  'volume': 0.0127223493665695,
  'fiducia': 0.1},
 'BNB': {'adx': 0.115147560229091,
  'atr': 0.1219564490252879,
  'close': 0.8375729858653909,
  'ema-100': 0.8870525182073266,
  'ema-200': 0.9182748567281164,
  'ema-50': 0.8591913185312718,
  'high': 0.8356817095772694,
  'low': 0.8428214210201668,
  'open': 0.8341523164493119,
  'rsi': 0.6002544031058116,
  'sma-100': 0.8720682330813073,
  'sma-200': 0.9214852602852838,
  'sma-50': 0.8521636947536342,
  'volume': 0.0024030637997116,
  'fiducia': 0.1},
 'BTC': {'adx': 0.1954733472661476,
  'atr': 0.2641607961662952,
  'close': 1.143

In [9]:
update_portfolio.set_portfolio()
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,0,0,0,0,0,0
BTC,0,0,0,0,0,0
BNB,0,0,0,0,0,0
SOL,0,0,0,0,0,0
ADA,0,0,0,0,0,0
LINK,0,0,0,0,0,0
XRP,0,0,0,0,0,0
LTC,0,0,0,0,0,0
XLM,0,0,0,0,0,0


In [10]:
Pt = portfolio_calculator.calculate(candle)
Pt

np.float64(100000.0)